# Install

In [1]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00


# Imports

In [2]:
import os
import psutil
import h5py
import os
import gc
import time
import copy
import psutil
import time
import numpy as np
from math import sqrt
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from typing import Any, Callable, List, Optional, Type, Union
import tensorflow as tf 
import tensorflow.keras as keras
import tensorflow_addons as tfa
from tensorflow.keras import layers


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# Setup Dataset

In [3]:
try:
  import google.colab
  COLAB_ENV = True
except:
  COLAB_ENV = False


if COLAB_ENV:
  data_filepath = "/" + os.path.join("content","drive","MyDrive","datasets", "ML4SCI_GSOC23")
  from google.colab import drive
  drive.mount('/content/drive')
else:
  data_filepath = "data"

photon_file = "SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"
electron_file = "SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"

model_save_folder = os.path.join(data_filepath, "models", "keras")
tensorboard_save_folder = os.path.join(data_filepath, "tensorboard")

photon_filepath = os.path.join(data_filepath, photon_file)
electron_filepath = os.path.join(data_filepath, electron_file)

electronFile = h5py.File(electron_filepath, "r")
photonFile = h5py.File(photon_filepath, "r")

electronX, electronY = np.array(electronFile.get("X")), np.array(electronFile.get('y'))
photonX, photonY     = np.array(photonFile.get("X")),   np.array(photonFile.get('y'))

mean_electron = np.mean(electronX.reshape(-1, 2), axis=0)
std_electron  = np.std(electronX.reshape(-1, 2), axis=0)
electronX = (electronX - mean_electron) / std_electron

mean_photon   = np.mean(photonX.reshape(-1, 2), axis=0)
std_photon    = np.std(photonX.reshape(-1, 2), axis=0)
photonX = (photonX - mean_electron) / std_electron

Mounted at /content/drive


## Train Test Split

In [4]:
training_size = 0.9
validation_size = 1. - training_size

size_of_dataset = len(electronX)
idxes = np.arange(size_of_dataset)
np.random.permutation(idxes)

idx_split = int(training_size * size_of_dataset)

train_examples_electron, train_labels_electron = electronX[:idx_split], electronY[:idx_split]
val_examples_electron, val_labels_electron = electronX[idx_split:], electronY[idx_split:]

train_examples_photon, train_labels_photon = photonX[:idx_split], photonY[:idx_split]
val_examples_photon, val_labels_photon     = photonX[idx_split:], photonY[idx_split:]

# Create Keras Dataset

In [5]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 500000

whole_dataset_X_train, whole_dataset_Y_train = np.concatenate([train_examples_electron, train_examples_photon], axis=0), \
                                               np.concatenate([train_labels_electron, train_labels_photon])

whole_dataset_X_val, whole_dataset_Y_val     = np.concatenate([val_examples_electron, val_examples_photon], axis=0), \
                                               np.concatenate([val_labels_electron, val_labels_photon], axis=0)

print(whole_dataset_X_train.shape)
train_dataset = tf.data.Dataset.from_tensor_slices((whole_dataset_X_train, whole_dataset_Y_train)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset  = tf.data.Dataset.from_tensor_slices((whole_dataset_X_val, whole_dataset_Y_val)).batch(BATCH_SIZE)

(448200, 32, 32, 2)


In [6]:
print(len(train_dataset))

7004


# Baseline

In [ ]:

def BasicNN():
    inputs = keras.Input(shape=(32,32, 2))

    x = layers.Flatten()(inputs)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(256, activation="relu")(x)
    outputs  = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

model = BasicNN()
#keras.utils.plot_model(model, show_shapes=True)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 flatten_5 (Flatten)         (None, 2048)              0         
                                                                 
 dense_10 (Dense)            (None, 256)               524544    
                                                                 
 dense_11 (Dense)            (None, 256)               65792     
                                                                 
 dense_12 (Dense)            (None, 1)                 257       
                                                                 
Total params: 590,593
Trainable params: 590,593
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(model_save_folder, "keras", "BasicNN.keras")),
]

clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-3,
    maximal_learning_rate=5e-3,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * len(train_dataset)
)
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(clr),
    loss="binary_crossentropy",
    metrics=["accuracy", "AUC"],
)
model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset)

Epoch 1/25
7004/7004 [==============================] - 28s 4ms/step - loss: 0.6242 - accuracy: 0.6573 - auc: 0.7070 - val_loss: 0.6041 - val_accuracy: 0.6843 - val_auc: 0.7337
Epoch 2/25
7004/7004 [==============================] - 28s 4ms/step - loss: 0.5984 - accuracy: 0.6878 - auc: 0.7416 - val_loss: 0.5976 - val_accuracy: 0.6867 - val_auc: 0.7435
Epoch 3/25
7004/7004 [==============================] - 27s 4ms/step - loss: 0.5855 - accuracy: 0.6997 - auc: 0.7570 - val_loss: 0.5839 - val_accuracy: 0.6998 - val_auc: 0.7578
Epoch 4/25
7004/7004 [==============================] - 28s 4ms/step - loss: 0.5636 - accuracy: 0.7170 - auc: 0.7796 - val_loss: 0.5771 - val_accuracy: 0.7084 - val_auc: 0.7662
Epoch 5/25
7004/7004 [==============================] - 27s 4ms/step - loss: 0.5468 - accuracy: 0.7290 - auc: 0.7954 - val_loss: 0.5858 - val_accuracy: 0.7048 - val_auc: 0.7616
Epoch 6/25
7004/7004 [==============================] - 27s 4ms/step - loss: 0.5466 - accuracy: 0.7291 - auc: 0.795

In [ ]:
print(model_save_folder)

/content/drive/MyDrive/datasets/ML4SCI_GSOC23/models


# Basic CNNs 
trainined only the best ones from pytorch script

## Experiment 1

In [ ]:
def getCNNSmall1():
  inputs = keras.Input(shape=(32,32, 2))

  x = layers.Conv2D(32, 3)(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(48, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(80, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.MaxPooling2D((2,2))(x)
  x = layers.Flatten()(x)

  outputs  = layers.Dense(1, activation="sigmoid")(x)
  return keras.Model(inputs, outputs)


model = getCNNSmall1()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 conv2d_17 (Conv2D)          (None, 30, 30, 32)        608       
                                                                 
 batch_normalization_17 (Bat  (None, 30, 30, 32)       128       
 chNormalization)                                                
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 28, 48)        13872     
                                                                 
 batch_normalization_18 (Bat  (None, 28, 28, 48)       192       
 chNormalization)                                          

In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(model_save_folder, "keras", "CNNSmall1.keras")),
]

clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=5e-4,
    maximal_learning_rate=3e-3,
    #scale_fn=lambda x: 1/(2.**(x-1)),
    scale_fn=lambda x:x,
    step_size=2 * len(train_dataset)
)
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(clr),
    loss="binary_crossentropy",
    metrics=["accuracy", "AUC"],
)
model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset)

Epoch 1/25
7004/7004 [==============================] - 114s 16ms/step - loss: 0.6191 - accuracy: 0.6642 - auc: 0.7173 - val_loss: 0.5921 - val_accuracy: 0.7017 - val_auc: 0.7594
Epoch 2/25
7004/7004 [==============================] - 109s 15ms/step - loss: 0.5752 - accuracy: 0.7077 - auc: 0.7687 - val_loss: 0.5946 - val_accuracy: 0.6901 - val_auc: 0.7646
Epoch 3/25
7004/7004 [==============================] - 109s 15ms/step - loss: 0.5605 - accuracy: 0.7191 - auc: 0.7834 - val_loss: 0.5539 - val_accuracy: 0.7266 - val_auc: 0.7903
Epoch 4/25
7004/7004 [==============================] - 108s 15ms/step - loss: 0.5477 - accuracy: 0.7288 - auc: 0.7955 - val_loss: 0.5456 - val_accuracy: 0.7316 - val_auc: 0.7979
Epoch 5/25
7004/7004 [==============================] - 108s 15ms/step - loss: 0.5477 - accuracy: 0.7286 - auc: 0.7957 - val_loss: 0.5852 - val_accuracy: 0.6930 - val_auc: 0.7604
Epoch 6/25
7004/7004 [==============================] - 108s 15ms/step - loss: 0.5572 - accuracy: 0.7220 

## Experiment 2

In [ ]:
def getCNNSmall5():
  inputs = keras.Input(shape=(32,32, 2))

  x = layers.Conv2D(32, 1)(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(32, 2)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(48, 2)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 5)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.Conv2D(64, 5)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.MaxPooling2D((2,2))(x)
  x = layers.Flatten()(x)

  outputs  = layers.Dense(1, activation="sigmoid")(x)
  return keras.Model(inputs, outputs)


model = getCNNSmall5()
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 32)        96        
                                                                 
 batch_normalization_22 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 activation_5 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 31, 31, 32)        4128      
                                                                 
 batch_normalization_23 (Bat  (None, 31, 31, 32)       128       
 chNormalization)                                          

In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(model_save_folder, "keras", "CNNSmall5.keras")),
]

clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=5e-4,
    maximal_learning_rate=3e-3,
    #scale_fn=lambda x: 1/(2.**(x-1)),
    scale_fn=lambda x:x,
    step_size=2 * len(train_dataset)
)
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(clr),
    loss="binary_crossentropy",
    metrics=["accuracy", "AUC"],
)
model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset)

Epoch 1/25
7004/7004 [==============================] - 146s 20ms/step - loss: 0.6121 - accuracy: 0.6739 - auc: 0.7281 - val_loss: 0.6809 - val_accuracy: 0.5609 - val_auc: 0.6785
Epoch 2/25
7004/7004 [==============================] - 140s 20ms/step - loss: 0.5708 - accuracy: 0.7121 - auc: 0.7732 - val_loss: 0.6069 - val_accuracy: 0.6747 - val_auc: 0.7517
Epoch 3/25
7004/7004 [==============================] - 140s 20ms/step - loss: 0.5550 - accuracy: 0.7240 - auc: 0.7888 - val_loss: 0.6089 - val_accuracy: 0.6574 - val_auc: 0.7705
Epoch 4/25
7004/7004 [==============================] - 139s 20ms/step - loss: 0.5433 - accuracy: 0.7321 - auc: 0.7997 - val_loss: 0.5410 - val_accuracy: 0.7337 - val_auc: 0.8011
Epoch 5/25
7004/7004 [==============================] - 139s 20ms/step - loss: 0.5428 - accuracy: 0.7322 - auc: 0.8002 - val_loss: 0.5706 - val_accuracy: 0.7223 - val_auc: 0.7909
Epoch 6/25
7004/7004 [==============================] - 139s 20ms/step - loss: 0.5509 - accuracy: 0.7265 

## Experiment 3

In [ ]:
def getCNNSmall8():
  inputs = keras.Input(shape=(32,32, 2))

  x = layers.Conv2D(32, 1)(inputs)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(32, 2)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(48, 2)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(64, 3)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(64, 5)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.Conv2D(64, 5)(x)
  x = layers.BatchNormalization()(x)
  x = layers.PReLU()(x)

  x = layers.MaxPooling2D((2,2))(x)
  x = layers.Flatten()(x)

  outputs  = layers.Dense(1, activation="sigmoid")(x)
  return keras.Model(inputs, outputs)


model = getCNNSmall8()
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 2)]       0         
                                                                 
 conv2d_35 (Conv2D)          (None, 32, 32, 32)        96        
                                                                 
 batch_normalization_35 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 p_re_lu_4 (PReLU)           (None, 32, 32, 32)        32768     
                                                                 
 conv2d_36 (Conv2D)          (None, 31, 31, 32)        4128      
                                                                 
 batch_normalization_36 (Bat  (None, 31, 31, 32)       128       
 chNormalization)                                          

In [ ]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint(os.path.join(model_save_folder, "keras", "CNNSmall8.keras")),
]

clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=5e-4,
    maximal_learning_rate=3e-3,
    #scale_fn=lambda x: 1/(2.**(x-1)),
    scale_fn=lambda x:x,
    step_size=2 * len(train_dataset)
)
 
model.compile(
    optimizer=tf.keras.optimizers.Adam(clr),
    loss="binary_crossentropy",
    metrics=["accuracy", "AUC"],
)
model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset)

Epoch 1/25
7004/7004 [==============================] - 219s 30ms/step - loss: 0.6082 - accuracy: 0.6779 - auc: 0.7333 - val_loss: 0.6298 - val_accuracy: 0.6578 - val_auc: 0.7295
Epoch 2/25
7004/7004 [==============================] - 213s 30ms/step - loss: 0.5686 - accuracy: 0.7134 - auc: 0.7758 - val_loss: 0.5646 - val_accuracy: 0.7144 - val_auc: 0.7805
Epoch 3/25
5289/7004 [=====================>........] - ETA: 49s - loss: 0.5559 - accuracy: 0.7229 - auc: 0.7880

# Efficient Net

In [7]:
from tensorflow.keras.applications import EfficientNetB0

model = EfficientNetB0(weights=None,input_shape=(32, 32, 2), classes=1, classifier_activation="sigmoid")
keras.utils.plot_model(model, show_shapes=True)

Output hidden; open in https://colab.research.google.com to view.

In [8]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 2)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 32, 32, 2)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 32, 32, 2)    5           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 33, 33, 2)    0           ['normalization[0][0]']          
                                                                                     

In [9]:
epochs = 25

BATCH_SIZE=64
steps_per_epoch = len(train_dataset) // BATCH_SIZE

clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=5e-3,
    maximal_learning_rate=7.5e-3,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * steps_per_epoch
)
optimizer = tf.keras.optimizers.Adam(clr)

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False,
                                            reduction="auto",
                                            name="binary_crossentropy"),
    metrics=["accuracy", "AUC"],
)
model.fit(
    train_dataset,
    epochs=epochs,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    validation_data=test_dataset)

Epoch 1/25
7004/7004 [==============================] - 340s 42ms/step - loss: 0.9521 - accuracy: 0.5088 - auc: 0.5128 - val_loss: 0.7027 - val_accuracy: 0.4987 - val_auc: 0.5078
Epoch 2/25
7004/7004 [==============================] - 296s 42ms/step - loss: 0.7137 - accuracy: 0.5286 - auc: 0.5421 - val_loss: 0.6820 - val_accuracy: 0.5625 - val_auc: 0.5987
Epoch 3/25
7004/7004 [==============================] - 303s 43ms/step - loss: 0.6510 - accuracy: 0.6222 - auc: 0.6569 - val_loss: 0.6646 - val_accuracy: 0.6207 - val_auc: 0.6613
Epoch 4/25
7004/7004 [==============================] - 301s 43ms/step - loss: 0.6269 - accuracy: 0.6575 - auc: 0.7008 - val_loss: 0.6173 - val_accuracy: 0.6746 - val_auc: 0.7350
Epoch 5/25
7004/7004 [==============================] - 291s 42ms/step - loss: 0.6138 - accuracy: 0.6734 - auc: 0.7174 - val_loss: 0.6300 - val_accuracy: 0.6517 - val_auc: 0.7294
Epoch 6/25
7004/7004 [==============================] - 286s 41ms/step - loss: 0.6489 - accuracy: 0.6235 